In [ ]:
import torch

rnn = torch.nn.RNN(input_size=2, hidden_size=3, num_layers=1)
input = torch.rand(5, 4, 2)  # seq_len序列长度, batch_size批量大小, input_size
hx = torch.randn(1, 4, 3)  # num_layers, batch_size, hidden_size

# 一次性输入整个序列
output1, hn = rnn(input=input, hx=hx)

# 依次输入序列中的每一个时刻
hn = hx
output2 = torch.zeros(output1.shape)
for i in range(input.shape[0]):
    output2[i], hn = rnn(input=input[i, :, :].unsqueeze(0), hx=hn)

# 比较输出结果
print(output1 == output2)